# Практическая работа к уроку №3

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

X_st = X.T.copy()
X_st[:, 1] = calc_std_feat(X_st[:, 1])
X_st[:, 2] = calc_std_feat(X_st[:, 2])
X_st[:, 3] = calc_std_feat(X_st[:, 3])

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [18]:
def calc_logloss(y, y_pred, eps=1e-8):
    err = - np.mean(y * np.log(y_pred + eps) + (1.0 - y) * np.log(1.0 - y_pred + eps))
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [19]:
def eval_model(X, y, iterations, alpha=1e-1):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1 / n * np.dot((y_pred - y).T, X))
        if i % 5000 == 0:
            print(i, W, err)
    return W

In [20]:
W = eval_model(X_st, y, iterations=50000)

5000 [ 1.53535465 -4.5908596  -3.5151729   9.41836954] 0.22241203642356439
10000 [ 2.48428253 -6.06769399 -5.34752279 13.29132815] 0.17913712050019812
15000 [ 3.17728368 -7.19574836 -6.70750681 16.21354578] 0.15473106260179312
20000 [ 3.72202873 -8.106698   -7.79591191 18.56784588] 0.13897337883019217
25000 [ 4.17132283 -8.87071374 -8.70905565 20.54705365] 0.12787657151420617
30000 [ 4.55467337 -9.52943786 -9.50100795 22.26234911] 0.11956367991544845
35000 [  4.89003336 -10.10931007 -10.20483817  23.78299785] 0.11304375327266888
40000 [  5.18905278 -10.62804616 -10.84208468  25.15475706] 0.10774689728535938
45000 [  5.45968273 -11.09804819 -11.42747681  26.40928677] 0.10332284168408164
50000 [  5.70757363 -11.5283176  -11.97148528  27.569326  ] 0.09954459557420335


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [21]:
def calc_pred_proba(W, X):
    z = np.dot(X, W)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

y_proba = calc_pred_proba(W, X_st)
y_proba

array([0.33472167, 0.00359485, 1.        , 0.04086419, 0.9489814 ,
       0.01841023, 1.        , 0.01510532, 0.63356234, 0.99999999])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [22]:
def calc_pred(W, X, treshhold=0.5):
    z = np.dot(X, W)
    y_pred_proba = sigmoid(z)
    y_pred = list(map(lambda x: 1 if x > treshhold else 0, y_pred_proba))
    return y_pred

y_pred = calc_pred(W, X_st)
y_pred

[0, 0, 1, 0, 1, 0, 1, 0, 1, 1]

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [23]:
def metrics(y, y_pred, print_res=True, beta=1):
    assert len(y) == len(y_pred), "dim y not equal y_pred"
    
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            if y[i] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if y[i] == 1:
                FN += 1
            else:
                FP += 1
    
    conf_matrix = pd.DataFrame(data=[[TP, FP], [FN, TN]],
                               columns=["y=1", "y=0"],
                               index=["y_pred=1", "y_pred=0"])
    accuracy = np.mean(y == y_pred)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = (1 + beta**2) / beta**2 * precision * recall / (precision + recall)

    if print_res:
        print(f"accuracy: {accuracy}\n"\
        f"precision: {precision}\n"\
        f"recall: {recall}\n"\
        f"f_score: {f_score}\n"\
        f"confusion matrix\n{conf_matrix}")

    return accuracy, precision, recall, f_score, conf_matrix

In [24]:
accuracy, precision, recall, f_score, conf_matrix = metrics(y, y_pred)

accuracy: 1.0
precision: 1.0
recall: 1.0
f_score: 1.0
confusion matrix
          y=1  y=0
y_pred=1    5    0
y_pred=0    0    5


6. Могла ли модель переобучиться? Почему?

Дать точного ответа на этот вопрос нельзя, так как у нас нет валидационной выборки, чтобы сравнить качество валидационной выборки с тестовой.

Но здесь все же модель явно переобучилась, так как мы получили 100% качество классификации.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [31]:
def eval_model_l1(X, y, iterations, l1_lambda, alpha=1e-3):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred) + np.sum(np.abs(W))
        W -= alpha * (1 / n * np.dot((y_pred - y).T, X)) \
            + l1_lambda * np.sign(W)
        if i % 5000 == 0:
            print(i, W, err)
    return W

In [33]:
W_l1 = eval_model_l1(X_st, y, iterations=50000, l1_lambda=0.0001)

5000 [ 9.50922352e-05 -4.15341080e-01 -1.23711029e-04  9.60498855e-01] 1.9342126735713396
10000 [-4.79995292e-05 -3.43497364e-01 -1.13023971e-04  7.86062787e-01] 1.705113949884076
15000 [ 7.42660207e-06 -2.45177754e-01 -2.05055477e-05  6.73509104e-01] 1.5109391890104842
20000 [-1.56190905e-05 -1.56847495e-01 -1.06862676e-04  5.82433132e-01] 1.3461785326640068
25000 [-8.03003614e-05 -8.09312036e-02  2.30905619e-05  5.05867837e-01] 1.206556660739628
30000 [-1.32862535e-05 -1.60870849e-02  5.16657263e-05  4.41131355e-01] 1.0882866657095036
35000 [-2.72305555e-05 -9.45264483e-05 -6.66833858e-05  3.97291337e-01] 1.0338741540191299
40000 [-8.75773588e-05  1.64795484e-05 -4.90014341e-05  3.82827002e-01] 1.0207014009477335
45000 [4.43387770e-05 8.31389818e-06 1.98684606e-05 3.78331168e-01] 1.0166364152445238
50000 [-8.71068977e-05 -1.60469205e-04  4.28635494e-05  3.76936797e-01] 1.0153231642369933


In [34]:
y_pred_l1 = calc_pred(W_l1, X_st)
accuracy, precision, recall, f_score, conf_matrix = metrics(y, y_pred_l1)

accuracy: 0.8
precision: 0.8
recall: 0.8
f_score: 0.8000000000000002
confusion matrix
          y=1  y=0
y_pred=1    4    1
y_pred=0    1    4


In [45]:
def eval_model_l2(X, y, iterations, l2_lambda, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations+1):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred) + np.sum(W **2)
        W -= alpha * (1 / n * np.dot((y_pred - y).T, X)) + l2_lambda * 2 * W
        if i % 5000 == 0:
            print(i, W, err)
    return W

In [46]:
W_l2 = eval_model_l2(X_st, y, iterations=50000, l2_lambda=0.0001)

5000 [ 0.16938244 -0.10717291  0.19332531  0.55315034] 1.0117814636036178
10000 [ 0.05543861 -0.06271139  0.06344662  0.23632606] 0.7186334205793807
15000 [ 0.01802815 -0.03377465  0.02948651  0.1346139 ] 0.6883155093796314
20000 [ 0.00585892 -0.01976303  0.02078086  0.10138403] 0.6849698717462976
25000 [ 0.00190546 -0.01376416  0.01857022  0.09030309] 0.6845780131579113
30000 [ 0.00062146 -0.0113516   0.01801488  0.08654102] 0.6845302083692851
35000 [ 0.00020448 -0.01041577  0.01787746  0.08524474] 0.6845242259134963
40000 [ 6.90848008e-05 -1.00608353e-02  1.78442099e-02  8.47927872e-02] 0.6845234656386425
45000 [ 2.51195637e-05 -9.92817880e-03  1.78364384e-02  8.46337708e-02] 0.6845233681211961
50000 [ 1.08443531e-05 -9.87908618e-03  1.78347254e-02  8.45774308e-02] 0.6845233555418693


In [47]:
y_pred_l2 = calc_pred(W_l2, X_st)
accuracy, precision, recall, f_score, conf_matrix = metrics(y, y_pred_l2)

accuracy: 0.8
precision: 0.8
recall: 0.8
f_score: 0.8000000000000002
confusion matrix
          y=1  y=0
y_pred=1    4    1
y_pred=0    1    4
